Django进阶实战-08
====
**模板的常用标签**

标签看起来像这样：`{％ tag ％}`。

标签比变量更复杂：有些在输出中创建文本，有些通过执行循环或逻辑来控制流，有些则将外部信息加载到模板中以供后来的变量使用。

某些标签需要开始和结束标签（即 {％tag％} ...标签内容... {％endtag％}）。

Django附带了大约24个内置模板标签。关于Django内置标签的完整信息，请参见[built-in tag reference](https://docs.djangoproject.com/en/2.0/ref/templates/builtins/#ref-templates-builtins-tags)

下是一些更常用的标签：

# block

定义一个块，这个块可以被子模板重写。参见[模板的继承](Django进阶实战10-模板的继承.ipynb)。

`{% block block_name %}{% endblock %}`

子模板中定义一个与父模板`block_name`相同的块，子模板将重写父模板中同名块的内容。

# extends

表示此模板扩展父模板的信号。

此标记以两种方式使用：

1. {％ extends“base.html” ％}（带引号）使用文字值“base.html”作为要扩展的父模板的名称。
2. {％ extends variable ％}使用变量的值。如果变量求值为字符串，Django将使用该字符串作为父模板的名称。如果变量求值为Template对象，Django将使用该对象作为父模板。

更多信息，请参见模板继承。

通常，模板名称相对于模板加载器的根目录。字符串参数也可以是以 ./ 或 ../ 开头的相对路径。 例如，假设以下目录结构：

```
dir1/
    template.html
    base2.html
    my/
        base3.html
base1.html
```
在 `template.html`中, 下面的路径是有效的：

- {% extends "./base2.html" %}
- {% extends "../base1.html" %}
- {% extends "./my/base3.html" %}

# filter

“过滤”多数时候是转换的意思，比如转换大小写。

通过一个或多个过滤器**过滤块**的内容。 可以使用管道指定多个过滤器，过滤器可以具有参数，就像在变量语法中一样。

请注意，该块包含 filter 和 endfilter 标记之间的所有文本。

示例用法：

```
{% filter force_escape|lower %}
    This text will be HTML-escaped, and will appear in all lowercase.
{% endfilter %}
```

**注意：**

escape and safe 过滤器是不可接受的参数。而是使用autoescape标记来管理模板代码块的自动转换。

# for

{% for %} 允许我们在一个序列上迭代。与Python的 for 语句类似，循环语法是 for X in Y ，Y是
要迭代的序列而X是在每一个特定的循环中使用的变量名称。每一次循环中，模板系统会渲染在 {% for %} 和{% endfor %} 之间的所有内容。

例如，要显示运动员列表中提供的运动员列表：

```
<ul>
{% for athlete in athlete_list %}
    <li>{{ athlete.name }}</li>
{% endfor %}
</ul>
```

## for 的反向迭代

给标签增加一个 reversed 使得该列表被反向迭代：

```
{% for athlete in athlete_list reversed %}
...
{% endfor %}
```

## 嵌套的 for 标签

可以嵌套使用 {% for %} 标签：

```
{% for athlete in athlete_list %}
    <h1>{{ athlete.name }}</h1>
    <ul>
        {% for sport in athlete.sports_played %}
            <li>{{ sport }}</li>
        {% endfor %}
    </ul>
{% endfor %}
```

## for 循环前执行 if 检查 

在执行循环之前先检测列表的大小是一个通常的做法，当列表为空时输出一些特别的提示。

```
{% if athlete_list %}

    {% for athlete in athlete_list %}
        <p>{{ athlete.name }}</p>
    
    {% endfor %}

    {% else %}
        <p>There are no athletes. Only computer programmers.</p>

{% endif %}
```

## for 的 empty 分句

for 标签支持一个可选的 {% empty %} 分句，通过它我们可以定义当列表为空时的输出内容 下面的例子与之前那个等价：

```
{% for athlete in athlete_list %}
    <p>{{ athlete.name }}</p>
    
    {% empty %}
        <p>There are no athletes. Only computer programmers.</p>

{% endfor %}
```

Django不支持退出循环操作。如果我们想退出循环，可以改变正在迭代的变量，让其仅仅包含需要迭代的项目。

同理，Django也不支持continue语句，我们无法让当前迭代操作跳回到循环头部。

在每个` {% for %}` 循环里有一个称为 forloop 的模板变量。这个变量有一些提示循环进度信息
的属性。

## forloop 变量

在每个 {% for %} 循环里有一个称为 forloop 的模板变量。这个变量有一些提示循环进度信息的属性。

> **forloop.counter** 是一个表示当前循环执行次数的整数计数器。这个计数器是从1开始的，所以在第一次循环时 forloop.counter 将会被设置为1。

```
{% for item in todo_list %}
    <p>{{ forloop.counter }}: {{ item }}</p>
{% endfor %}
```
> **forloop.counter0** 类似于 forloop.counter ，但是它是从0计数的。第一次执行循环时这个变量会被设置为0。

> **forloop.revcounter** 表示循环中剩余项的整型变量。在循环初次执行时 forloop.revcounter 将被
设置为序列中项的总数。最后一次循环执行中，这个变量将被置1。

> **forloop.revcounter0** 类似于 forloop.revcounter ，但它以0做为结束索引。 在第一次执行循环时，
该变量被置为序列的项的个数减1。

> **forloop.first** 是一个布尔值，如果该迭代是第一次执行，那么它被置为 "",在下面的情形中这个变量是
很有用的：

```
{% for object in objects %}

    {% if forloop.first %}
        <li class="first">{% else %}<li>
    {% endif %}
    
    {{ object }}
    </li>

{% endfor %}r %}
```

> **forloop.last** 是一个布尔值；在最后一次执行循环时被置为True。 一个常见的用法是在一系列的链接之间放置管道符（|）

```
{% for link in links %}
    {{ link }}
    {% if not forloop.last %} 
        | 
    {% endif %}
{% endfor %}
```

上面的模板可能会产生如下的结果：

**Link1 | Link2 | Link3 | Link4**

另一个常见的用途是为列表的每个单词的加上逗号。
```
{% for p in places %}
    {{ p }}
    {% if not forloop.last %}
        , 
    {% endif %}
{% endfor %}
```

**forloop.parentloop** 是一个指向当前循环的上一级循环的 forloop 对象的引用（在嵌套循环的情况下）。

示例：
```
{% for country in countries %}
    <table>
        {% for city in country.city_list %}
        <tr>
            <td>Country #{{ forloop.parentloop.counter }}</td>
            <td>City #{{ forloop.counter }}</td>
            <td>{{ city }}</td>
        </tr>
        {% endfor %}
    </table>
{% endfor %}
```

forloop 变量仅仅能够在循环中使用。 在模板解析器碰到{% endfor %}标签后，forloop就不可访问了。

**Context和forloop变量**

在一个 {% for %} 块中，已存在的变量会被移除，以避免 forloop 变量被覆盖。

Django会把这个变量移动到forloop.parentloop 中。通常我们不用担心这个问题，但是一旦我们在模板中定义了forloop 这个变量（当然我们反对这样做），在 {% for %} 块中它会在 forloop.parentloop 被重新命名。

# if

计算变量的值，如果结果为"true"，就显示标签块的内容，否则就不显示。

```
{% if today_is_weekend %}
    <p>Welcome to the weekend!</p>
{% endif %}
```

## if 的 else 标签

if标签可以嵌套一个else标签：

```
{% if today_is_weekend %}
        <p>Welcome to the weekend!</p>
    {% else %}
        <p>Get back to work.</p>
{% endif %}
```

## if 的 elif 标签

```
{% if athlete_list %}
    Number of athletes: {{ athlete_list|length }}
    
    {% elif athlete_in_locker_room_list %}
        Athletes should be out of the locker room soon!
    
    {% else %}
        No athletes.

{% endif %}
```
**注意：嵌套的 else 和 elif 标签没有结束标签。而每个 if 标签都必须用 endif 关闭。**

### Python 的“真值”

在Python和Django模板系统中，以下这些对象相当于布尔值的False：

- 空列表([] )
- 空元组(() )
- 空字典({} )
- 空字符串('' )
- 零值(0 )
- 特殊对象None
- 对象False（很明显）

提示：你也可以在自定义的对象里定义他们的布尔值属性(这个是python的高级用法)。

除以上几点以外的所有东西都视为` True `

## if 接受布尔操作符

if 标签可以使用 and，or, not 测试多个变量或非给定变量：

```
{% if athlete_list and coach_list %}
    Both athletes and coaches are available.
{% endif %}

{% if not athlete_list %}
    There are no athletes.
{% endif %}

{% if athlete_list or coach_list %}
    There are some athletes or some coaches.
{% endif %}

{% if not athlete_list or coach_list %}
    There are no athletes or there are some coaches.
{% endif %}

{% if athlete_list and not coach_list %}
    There are some athletes and absolutely no coaches.
{% endif %}
```

在同一个标签中允许使用 and 和 or 子句，and 比 or 有更高的优先级：

`{% if athlete_list and coach_list or cheerleader_list %}`

将被解释为：

`if (athlete_list and coach_list) or cheerleader_list`

但在 if 标记中使用实际括号是无效的语法。如果需要它们来指示优先级，则应使用嵌套的if标记。

## if 的比较操作符

if tags may also use the operators ==, !=, <, >, <=, >=, in, not in, is, and is not which work as follows:

if 标签也以使用比较操作符：

**==, !=, <, >, <=, >=, in, not in, is, is not**

### ==，等于
```
{% if somevar == "x" %}
  This appears if variable somevar equals the string "x"
{% endif %}
```

### !=，不等于
```
{% if somevar != "x" %}
  This appears if variable somevar does not equal the string "x",
  or if somevar is not found in the context
{% endif %}
```

### <，小于
```
{% if somevar < 100 %}
  This appears if variable somevar is less than 100.
{% endif %}
```

### >，大于 
```
{% if somevar > 0 %}
  This appears if variable somevar is greater than 0.
{% endif %}
```

### <=，小于等于 
```
{% if somevar <= 100 %}
  This appears if variable somevar is less than 100 or equal to 100.
{% endif %}
```

### >=，大于等于
```
{% if somevar >= 1 %}
  This appears if variable somevar is greater than 1 or equal to 1.
{% endif %}
```

### in，包含在里面

这个操作符支持很多Python容器，测试容器中有否有给定的值。示例：
```
{% if "bc" in "abcdef" %}
  This appears since "bc" is a substring of "abcdef"
{% endif %}

{% if "hello" in greetings %}
  If greetings is a list or set, one element of which is the string
  "hello", this will appear.
{% endif %}

{% if user in users %}
  If users is a QuerySet, this will appear if user is an
  instance that belongs to the QuerySet.
{% endif %}
```

### not in，不包含里面。

这是 in 操作符的否定形式。

### is，是

对象的身份一致性。测试两个值是否为同一个对象。例：

```
{% if somevar is True %}
  This appears if and only if somevar is True.
{% endif %}

{% if somevar is None %}
  This appears if somevar is None, or if somevar is not found in the context.
{% endif %}
```

### is not，不是

否定对象身份的一致性，测试两个值不是同一个对象。is 操作符的否定形式：

```
{% if somevar is not True %}
  This appears if somevar is not True, or if somevar is not found in the
  context.
{% endif %}

{% if somevar is not None %}
  This appears if and only if somevar is not None.
{% endif %}
```

## if 中使用 Filters
You can also use filters in the if expression. For example:

还可以在 if 表达式中使用过滤器。

{% if messages|length >= 100 %}
   You have lots of messages today!
{% endif %}

## 复合表达式

可以使用上述运算符构造复杂表达式。对于这样的表达式，最重要的就是要清晰地知道各运算符在表达式中的优先级。

下面是运算符的优先级从低到高的排列：

- or
- and
- not
- in
- ==, !=, <, >, <=, >=

(这完全是Python的语法).下面是 if 标签中的复合表达式使用示例：

`{% if a == b or c == d and e %}`

将被解释为:
`(a == b) or ((c == d) and e)`

如果需要不同的优先级，就发使用嵌套的 if 标签。有时对于不清楚运算符优先级规则的人，使用嵌套的 if 反而是更清晰的方法。

比较运算符一能像Python或数学运算中那样“链式”比较。例如：

`{% if a > b > c %}`  (错误的)

应该这样：

`{% if a > b and b > c %}`

## ifequal/ifnotequal

这是过时的用法，在最新的Django中已被比较运算符代替。

## ifchanged

检测值与最后一次循环迭代时是否一样，即在上一次循环迭代后值是否有改变。

{% ifchanged %} 块标签用在循环内部。有两个可能的用途。

Checks its own rendered contents against its previous state and only displays the content if it has changed. For example, this displays a list of days, only displaying the month if it changes:

> 1.检查其自身呈现的内容与其先前的状态，并仅显示内容（如果已更改）。例如，下面代码，会显示一个天数列表，如果它发生了变化，就增加显示月份（没有变化就不显示月份。）：

```
<h1>Archive for {{ year }}</h1>

{% for date in days %}
    {% ifchanged %}
        <h3>{{ date|date:"F" }}</h3>
    {% endifchanged %}
    
    <a href="{{ date|date:"M/d"|lower }}/">{{ date|date:"j" }}</a>
    
{% endfor %}
```

> 2.如果给定一个或多个变量，检查是否有任何变化。下面的示例显示每次更改时的日期，如果小时或日期有变化，还同时显示小时：

```
{% for date in days %}
    {% ifchanged date.date %} 
        {{ date.date }} 
    {% endifchanged %}
    
    {% ifchanged date.hour date.date %}
        {{ date.hour }}
    {% endifchanged %}
    
{% endfor %}
```

ifchanged 标记还可以采用可选的 {％else％} 子句，如果值未更改，将显示该子句：
```
{% for match in matches %}
    <div style="background-color:
        {% ifchanged match.ballot_id %}
            {% cycle "red" "blue" %}
        {% else %}
            gray
        {% endifchanged %}
    ">{{ match }}</div>
{% endfor %}
```

# include
加载模板并使用当前上下文呈现它。 这是一种在模板中“包含”其他模板的方法。

模板名称可以是单引号或双引号中的变量或硬编码（带引号）字符串。

下面的示例“包含”模板 “foo / bar.html” 的内容：

`{% include "foo/bar.html" %}`

通常，模板名称相对于模板加载器的根目录。字符串参数也可以是以 ./ 或 ../ 开头的相对路径，参考 extends 标标签中关于目录的描述。

下面的示例包括模板的内容，该模板的名称包含在变量template_name中：

`{% include template_name %}`

变量也可以是一个对象，该对象有一个接受 context 的 render()方法。允许您在上下文中引用已编译的模板。

被包含的模板在包含它的模板的上下文中呈现。此示例生成输出“Hello，John！”：

Context: 变量 person 设置为 "John"，变量 greeting 设置为 "Hello".

** 模板 **:
`{% include "name_snippet.html" %}`

**name_snippet.html 模板**：
`{{ greeting }}, {{ person|default:"friend" }}!`

需要在“视图--urls--html模板文件”下测试。注意模板文件中，"name_snippet.html" 引用路径。

在'pyhotn manage.py shell'模式下，只能直接渲染‘name_snippet.html’模板，尝试渲染`{% include "name_snippet.html" %}`是不可行的，因为父模板与子模板是分开渲染的，两者间不能共享状态。

可以使用关键字参数向模板传递附加的context：

`{% include "name_snippet.html" with person="Jane" greeting="Hello" %}`

如果只想使用提供的变量渲染上下文（甚至根本没有变量），请使用唯一选项。 包含的模板没有其他变量可用：

`{% include "name_snippet.html" with greeting="Hi" only %}`

如果包含的模板在呈现时导致异常（包括它是否缺失或具有语法错误），则行为会因模板引擎的调试选项而异（如果未设置，则此选项默认为DEBUG的值）。

- 打开调试模式时，将引发TemplateDoesNotExist或TemplateSyntaxError等异常。
- 当关闭调试模式时，{％include％}会向django.template记录器发出警告，但在呈现包含的模板时会发生异常并返回空字符串。

**注意**

include标记应该被视为“渲染此子模板并包含HTML”的实现，而不是“解析此子模板并将其内容包含在内，就好像它是父类的一部分”。**这意味着包含的模板之间没有共享状态 - 每个包含都是完全独立的渲染过程。**

在包含块之前对块进行评估。 这意味着包含来自另一个块的块的模板将包含已经评估和渲染的块 - 而不是可以被扩展模板覆盖的块。

# load

加载自定义模板标记集。

例如，以下模板将加载在package包中的somelibrary和otherlibrary中注册的所有标记和过滤器：

`{% load somelibrary package.otherlibrary %}`

您还可以使用from参数从库中有选择地加载单个过滤器或标记。在此示例中，将从somelibrary库中加载名为foo和bar的模板标记/过滤器：

`{% load foo bar from somelibrary %}`

这在使用第三方模板库，如有名的bootstrap模板库，会经常用到。